In [2]:
!pip install torch torchvision torchaudio lightning

In [11]:
a = 1
print(a)

1


In [15]:
import cv2
import mediapipe as mp
import numpy as np
from datetime import datetime
import os

SAVE_DIR = r"C:\Users\Users\eye_captures"
os.makedirs(SAVE_DIR, exist_ok=True)

# Mediapipe 초기 설정
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    refine_landmarks=True,
    max_num_faces=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 눈 랜드마크 인덱스 (정확한 좌우 기준)
RIGHT_EYE_IDX = [33, 133, 160, 159, 158, 157, 173, 246]   # 사람의 오른쪽 눈 (화면에서는 왼쪽)
LEFT_EYE_IDX = [362, 263, 387, 386, 385, 384, 398, 466]   # 사람의 왼쪽 눈 (화면에서는 오른쪽)

# 눈 이미지 추출 함수
def extract_eye(image, landmarks, eye_indices, padding=20):
    ih, iw, _ = image.shape
    eye_points = np.array([(int(landmarks[i].x * iw), int(landmarks[i].y * ih)) for i in eye_indices])
    x, y, w, h = cv2.boundingRect(eye_points)

    x = max(x - padding, 0)
    y = max(y - padding, 0)
    w = min(w + padding * 2, iw - x)
    h = min(h + padding * 2, ih - y)

    eye_img = image[y:y+h, x:x+w]
    return eye_img

# 웹캠 시작
cap = cv2.VideoCapture(0)
print("웹캠 실행 중... 's' 키를 누르면 눈을 저장하고, 'q' 키를 누르면 종료됩니다.")

left_eye_img = None
right_eye_img = None

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("카메라 프레임을 가져올 수 없습니다.")
        break

    # RGB 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    # 눈 영역 추출
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark
            try:
                left_eye_img = extract_eye(frame, landmarks, LEFT_EYE_IDX)
                right_eye_img = extract_eye(frame, landmarks, RIGHT_EYE_IDX)

                # 눈 이미지 실시간 출력
                if left_eye_img.size > 0:
                    cv2.imshow("Left Eye", left_eye_img)
                if right_eye_img.size > 0:
                    cv2.imshow("Right Eye", right_eye_img)
            except Exception as e:
                print("눈 추출 중 오류 발생:", e)
                continue

    # 원본 영상 표시
    cv2.imshow("Webcam", frame)

    key = cv2.waitKey(1) & 0xFF

    # 눈 저장: 's' 키
    if key == ord('s') and left_eye_img is not None and right_eye_img is not None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%f")
        left_path = f"left_eye_{timestamp}.png"
        right_path = f"right_eye_{timestamp}.png"

        # ✅ 눈 이미지 90x90으로 리사이즈
        left_resized = cv2.resize(left_eye_img, (90, 90))
        right_resized = cv2.resize(right_eye_img, (90, 90))

        # ✅ 저장
        cv2.imwrite(os.path.join(SAVE_DIR, left_path), left_resized)
        cv2.imwrite(os.path.join(SAVE_DIR, right_path), right_resized)

        print(f"눈 저장 완료: {left_path}, {right_path}")

    # 종료: 'q' 키
    elif key == ord('q'):
        print("종료합니다.")
        break

# 종료 처리
cap.release()
cv2.destroyAllWindows()

웹캠 실행 중... 's' 키를 누르면 눈을 저장하고, 'q' 키를 누르면 종료됩니다.
눈 저장 완료: left_eye_20250715_104019130640.png, right_eye_20250715_104019130640.png
눈 저장 완료: left_eye_20250715_104020218138.png, right_eye_20250715_104020218138.png
눈 저장 완료: left_eye_20250715_104023353189.png, right_eye_20250715_104023353189.png
종료합니다.


In [12]:
import os
print("현재 작업 디렉토리:", os.getcwd())

현재 작업 디렉토리: C:\Windows\System32
